# Project Rosalind

Project [Rosalind](http://rosalind.info/problems/list-view/) solutions in Dyalog APL.

By Stefan Kruger

In [28]:
⎕IO←0    ⍝ Index origin 0
⎕PP←34   ⍝ Numeric print precision
⎕FR←1287 ⍝ IEEE 754-2008 128-bit decimal floating-point operations
]box on -s=max
]rows on
hc←⎕SE.SALT.Load'HttpCommand'
'pmat'⎕CY'dfns'

┌→────────────────┐
│Was ON -style=max│
└─────────────────┘

┌→─────┐
│Was ON│
└──────┘

Some utility routines

In [29]:
file←{⊃⎕NGET('/Users/stefan/work/notebooks/rosalind/data/',⍵)1}

In [30]:
assert←{⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}
print←'¯'⎕R'-'⍤⍕⍤1⊢

In [31]:
]dinput
FASTA←{ ⍝ Read a DNA, RNA or protein string in the FASTA format
    file ← '/Users/stefan/work/notebooks/rosalind/data/',⍵
    flat ← (∊'>'=d)⊂∊d←⊃⎕NGET file 1
    names ← ⊃¨'^>(Rosalind_\d+)'⎕S'\1'¨flat
    names ('^>Rosalind_\d+'⎕R''¨flat)
}

In [32]:
]dinput
FASTA2←{ ⍝ Read a DNA, RNA or protein string in the FASTA format
    dd←file⍵
    n←⎕NS⍬
    n.(names strings)←⍬⍬
    _←{n.names,←⊂⊃⍵⋄n.strings,←,/1↓⍵⋄⍬}¨dd⊂⍨'>'∘=⊃¨dd
    n.names←1∘↓¨n.names
    n
}

In [37]:
]dinput
UniProt←{ ⍝ Fetch a protein definition by uniprot_id
    keys←,⊆⍵
    urls←(⊂'https://www.uniprot.org/uniprot/')∘.,keys∘.,⊂'.fasta'
    r←hc.Get¨urls
    {'\n'⎕R''⍠('Mode' 'D')('DotAll' 1)⊢⊃,/(⎕UCS 10)(=⊂⊢)⍵}¨r.Data
}

In [5]:
compl_dna←{'TAGC'['ATCG'⍳⍵]}
rc←⊖∘compl_dna

## Transcribing DNA into RNA
http://rosalind.info/problems/rna/

In [215]:
rna ← 'U'@('T'∘=) ⍝ T becomes U

In [217]:
assert 'GAUGGAACUUGACUACGUAAAUU'≡⎕←rna 'GATGGAACTTGACTACGTAAATT'

GAUGGAACUUGACUACGUAAAUU

## Complementing a Strand of DNA

http://rosalind.info/problems/revc/

In [218]:
revc←{'TAGC'['ATCG'⍳⍵]}

In [219]:
assert 'TTTTGGGCCA'≡⎕←revc 'AAAACCCGGT'

TTTTGGGCCA

## Rabbits and Recurrence Relations
http://rosalind.info/problems/fib/

In [220]:
fib←{g←⍺⋄{⍺←0 1⋄⍵=0:⊃⍺⋄(1↓⍺,⍺[1]+g×⍺[0])∇⍵-1}⍵}

In [221]:
assert 19=⎕←3 fib 5
assert 14415648500221=⎕←5 fib 31

19

14415648500221

## Computing GC Content
http://rosalind.info/problems/gc/

In [204]:
n←FASTA2'rosalind_gc.txt'
gc←⍎7⍕100×(≢⍤⊢÷⍨+/⍤∊∘'GC')¨n.strings
(⊃⍒gc)⊃¨n.names gc

Rosalind_7042 50.9933775

## Counting Point Mutations
http://rosalind.info/problems/hamm/

In [222]:
assert 469=⎕←+/≠⌿↑file'rosalind_hamm.txt'

469

## Mendel's First Law
http://rosalind.info/problems/iprb/

Some maths I can't take credit for.

In [135]:
]dinput
iprb←{
    (k m n)←⍵
    pop←+/⍵ ⋄ mp←m÷pop ⋄ np←n÷pop
    dom←k÷pop                                             ⍝ Homozygous dominant first
    het←(mp×k÷pop-1)+(mp×0.75×(m-1)÷pop-1)+mp×0.5×n÷pop-1 ⍝ Heterozygous first
    rec←(np×k÷pop-1)+np×0.5×m÷pop-1                       ⍝ Recessive first
    dom+het+rec
}

In [136]:
iprb 2 2 2
iprb 17 26 20

0.7833333333333333333333333333333333

0.727982590885816692268305171530978

## Translating RNA into Protein
http://rosalind.info/problems/prot/

See https://en.wikipedia.org/wiki/Genetic_code#/media/File:3D_Genetic_Code.jpg

In [29]:
codon←0 2 1⍉⍉4 4 4⍴'FLIVFLIVLLIVLLMVSPTASPTASPTASPTAYHNDYHND.QKE.QKECRSGCRSG.RRGWRRG'
test←'AUGGCCAUGGCGCCCAGAACUGAGAUCAAUAGUACCCGUAUUAACGGGUGA'

In [30]:
prot←{enc←0 1 2 3['UCAG'⍳⍵]⋄¯1↓⍺[⊂⍤1⊢(3÷⍨≢enc) 3⍴enc]}

In [31]:
⊢r←codon prot test
assert r≡'MAMAPRTEINSTRING'

MAMAPRTEINSTRING

In [32]:
data←file'rosalind_prot.txt'
r←codon prot (⊃data)
54↑r ⍝ String is long. Show the first 50 or so letters only
assert 'MWESKYAVSANIAWSYINHADVRLSSDGRYEVPPNFRQAIKLIMPFHSAGLYYR'≡54↑r

MWESKYAVSANIAWSYINHADVRLSSDGRYEVPPNFRQAIKLIMPFHSAGLYYR

## Finding a Motif in DNA
http://rosalind.info/problems/subs/

In [33]:
sub←⍸⍷

In [34]:
data←file'rosalind_subs.txt'
r←1+⊃sub⍨/data ⍝ 1-index!
9↑r ⍝ List is long....
assert 113 131 138 154 321 355 384 407 414≡9↑r 

113 131 138 154 321 355 384 407 414

## Consensus and Profile
http://rosalind.info/problems/cons/

In [167]:
data←FASTA2'rosalind_cons.txt'
profile ← ⍉↑(+/'ACGT'⍷⍤0 1⊢)¨↓⍉↑data.strings      ⍝ TODO: this could be better
consensus ← 'ACGT'[⊃∘⍒⍤1⊢⍉profile]

⍝ Format the output as per problem description
len ← 1⊃⍴∊⍤1⊢table ← (⍪'A: ' 'C: ' 'G: ' 'T: '),⍕profile
(len↑consensus)⍪∊⍤1⊢table

CAAAACACGGAGAAACGCAGACGAAGCACTGTACAAAGTACTGAACAAGAGCAAATTTTTAAGCTGAAGCGAACCTAGATAGCCGTGAGGCAGACCCCCTCACAGAGAAAACATACTACAATCCATACCACGTCCGAAGAAAACCCGCAAATCTGCGTCTTGTTAGAAGCAAACAAATACGACCGAACAGATTAAGACCGTCACAACGTGAGTAACCCTGCAAGGAGAGCAACCGCTGTCACCGTCACAGAAGACTCAGAGCAAACCGCCCCGATAACCCGAATGGTAGAAACTAGTGGACTTAACTTCTGAATTACAATTTGGTCGGGAGAAAGACAGAAAACATGGTGAACCCACTCCGGCACCTGGGCCGGGCAGAACGTAATGGAAGCAGCGTACGCGTATAAATGCACAAGCAACACAGTACACATATCTCATCTCAACGGAAACGGGGTTGCAATGCTTCTTTACAAAATTCTACAGCGACATCGGAATTAACGCGGAACAGGATTGCACCAACTGACGTCCGAAGCAAGAGTGACAGGCAGACTAACAAGATTGGAAAGACCATTCGCGGTAAAAAAGTGGTGGGATGTAGGAATCGCGAATATCTCTTTGAAACAACATCGAAGTGGCCACCAAAACTACCCAAGACGCGAAGGACTCTCAAGGGCTATCAGTTCAGAAAGGCGTACCTGGCCACATTTTAGGCACCCGAAACAAAAAACCAAACACTCAAGGAGTGCCAGCCAGAATACAACATGGTTTTCTTTACATGAAAGAAAAACAACTAGCCAAGCCTAAAATAACAACTGTCTTGTTTTGCAACCACCTTCCGATAAGGAAGAGTAAGTCGATGTCCTGGTGAATTTCAACGCTTAACGCACAAAACCCGCAATACCCTAAGTCCTAGCATACACCTAAAGATAGGGCCCCCCGCATGCCGCGCAGGGTGGATCACTCGTGCAAAGACCCTTTGGTGAACGAGCAGACAACACA 
A: 2 5 5 3 6 2 4 0 0 2 3 2 3 4 4 3 2 3 5 3 3 2 1 4 3 1 2 4 1 1 3 1 3 2 4 5 3 1 3 3 1 3 2 5 4 0 4 3 2 3 2 3 4 4 4 3 3 2 2 3 4 3 3 1 1 1 3 3 0 0 2 3 3 2 3 1 3 2 3 2 4 2 2 4 2 2 1 4 3 3 2 4 3 5 3 2 2 2 3 1 2 4 0 4 3 4 2 4 4 5 3 2 5 3 4 2 2 6 0 3 5 2 2 3 4 3 4 2 2 4 1 2 2 2 2 2 3 4 2 6 3 5 4 1 1 2 1 0 6 5 4 3 2 3 3 1 2 3 2 3 2 1 0 2 5 2 3 4 1 2 4 4 6 2 5 3 5 1 3 0 2 4 1 1 1 5 3 4 3 4 5 2 1 3 3 1 4 2 1 3 4 3 3 2 4 3 2 1 2 1 4 3 2 5 4 2 3 0 3 3 1 5 4 2 2 4 1 4 0 2 5 4 2 1 0 0 0 2 1 2 3 1 2 2 2 2 3 2 4 3 3 4 2 3 2 2 1 3 2 3 3 2 3 4 5 3 1 1 2 2 4 3 0 3 1 4 4 2 1 2 3 3 3 0 2 1 2 4 2 4 3 4 0 2 4 3 1 2 2 4 2 1 1 4 6 1 1 1 1 1 2 4 4 2 3 4 2 3 4 3 1 3 3 1 1 1 2 3 2 3 3 3 3 4 2 4 3 3 1 4 6 3 4 3 3 2 3 2 1 0 5 4 2 3 2 3 1 2 2 2 2 2 3 4 2 2 1 2 3 1 1 4 2 2 1 2 4 2 4 4 2 3 3 3 4 2 3 2 4 4 3 2 5 3 1 3 3 4 1 0 1 1 3 3 2 5 3 4 0 2 2 5 2 4 3 4 2 5 4 1 4 2 4 3 1 6 1 3 2 4 0 3 1 2 3 1 4 1 1 0 3 4 4 1 1 1 5 4 5 2 2 1 1 2 1 2 2 3 3 3 1 1 2 3 3 1 2 2 1 3 2 4 5 3 3 2 2 3 2 3 2 6 2 1 2 4 3 4 3 2 1 2 4 4 3 1 4 3 3 0 1 3 3 5 3 2 4 2 2 4 1 2 3 1 7 2 1 3 3 0 3 1 4 3 3 2 1 1 1 4 4 1 1 4 3 2 4 1 1 1 6 0 4 1 2 2 5 3 4 1 2 5 4 2 4 4 1 5 4 3 0 2 5 4 5 3 4 2 2 3 2 3 1 2 2 1 0 2 4 4 5 3 3 3 2 1 2 2 1 1 0 2 4 1 2 3 4 3 3 4 4 2 2 3 2 3 4 4 1 3 1 2 2 1 3 3 2 2 3 3 4 2 3 5 2 4 2 1 3 4 4 2 2 2 2 2 1 3 1 2 3 4 3 4 2 1 3 1 2 2 4 4 2 3 2 2 3 2 5 3 0 1 4 1 2 0 1 2 5 5 1 1 2 3 3 4 3 2 4 2 1 2 1 3 2 3 4 3 1 2 2 2 1 3 2 1 3 3 2 1 1 4 3 5 3 1 3 2 4 2 2 2 5 2 1 2 2 5 4 4 2 5 4 3 5 4 4 2 3 4 3 6 2 3 2 1 2 5 4 3 1 4 2 0 0 3 2 3 2 3 2 5 2 4 4 3 3 2 3 3 2 3 2 1 1 3 1 2 0 2 2 2 2 4 3 5 3 2 4 4 4 2 3 3 4 3 3 3 3 5 3 2 4 2 1 2 4 4 3 2 2 1 4 4 4 4 3 4 4 1 5 4 1 2 3 1 1 3 3 2 3 3 1 3 3 3 3 4 2 2 6 3 2 1 2 2 1 3 3 3 6 4 2 2 3 4 1 3 1 1 3 3 2 0 2 3 4 1 2 2 1 0 2 3 3 0 3 4 4 1 2 1 2 3 4 2 2 1 2 1 4 4 1 0 2 3 3 6 4 3 4 2 3 1 2 0 4 3 3 4 3 1 2 2 4 3 2 4 2 1 2 3 3 3 3 3 5 3 4 3 1 2 4 6 4 2 3 3 4 2 3 3 2 2 1 1 2 3 2 1 4 1 2 2 3 3 2 1 1 5 1 2 3 2 2 2 3 1 3 3 2 2 2 0 2 2 2 3 4 4 2 5 1 2 1 1 2 1 1 2 3 3 4 4 2 2 4 2 3 6 1 3 2 5 4 0 4 2 3
C: 4 2 1 2 2 4 1 5 1 3 3 2 3 0 4 4 2 4 1 2 3 3 3 2 2 3 3 2 3 2 2 1 1 3 0 3 3 4 0 3 3 1 2 2 3 4 3 2 1 1 0 4 3 3 3 2 0 2 3 1 2 3 0 5 3 3 2 3 4 5 1 3 3 4 4 1 3 2 3 0 3 2 4 5 1 3 2 2 0 2 4 3 1 1 5 4 3 3 4 2 3 4 4 3 3 3 1 2 2 2 3 4 0 1 2 3 1 1 4 3 1 3 3 4 0 0 2 3 4 1 3 2 2 3 5 2 2 2 1 3 2 2 4 3 5 3 1 4 1 2 0 0 4 1 1 4 2 2 4 0 1 2 3 2 1 1 3 1 3 4 1 1 2 3 1 3 1 0 2 6 2 2 4 4 1 1 3 5 2 1 3 3 2 2 2 0 2 4 4 2 1 4 3 5 3 1 4 2 3 2 3 1 3 3 2 4 4 5 1 1 4 1 1 2 2 2 2 3 1 3 1 3 4 4 2 4 2 2 2 5 1 4 3 3 3 4 2 4 4 1 2 4 3 3 4 1 4 2 2 3 2 4 2 3 2 4 4 1 3 3 5 4 2 3 3 1 1 3 4 4 2 3 3 2 2 3 1 2 1 2 2 2 5 1 1 0 3 0 1 1 4 2 2 3 1 3 3 2 6 2 1 1 2 3 2 2 4 3 0 1 2 3 1 1 2 4 1 2 3 2 0 2 3 1 2 1 4 3 3 2 2 3 1 4 3 3 0 2 2 1 1 2 3 4 3 2 3 2 3 4 2 1 4 1 3 5 2 1 1 3 4 5 1 3 3 5 3 3 0 3 3 1 1 3 1 1 0 2 2 2 0 3 2 1 5 2 0 1 3 1 5 0 2 3 1 3 3 2 3 2 5 0 5 1 3 1 4 2 2 4 2 5 1 3 2 0 4 2 3 2 1 3 2 3 3 4 2 1 6 4 4 2 3 4 3 1 3 4 0 4 2 1 2 3 3 1 1 4 2 3 0 3 3 2 2 4 2 1 1 2 5 2 2 3 2 1 3 4 2 2 5 3 2 5 0 1 5 2 1 3 2 3 1 2 1 3 3 2

In [168]:
assert 'CAAAACACGGAGAAACGCAGACGAAGCA'≡28↑consensus

## Mortal Fibonacci Rabbits
http://rosalind.info/problems/fibd/

## Overlap Graphs
http://rosalind.info/problems/grph/

In [174]:
data←FASTA2'rosalind_grph-2.txt'
m ← ∘.{(¯3↑⍺)≡(3↑⍵)} ⍨data.strings
(0 0⍉m) ← 0
r ← (2÷⍨≢pairs) 2⍴pairs ← data.names[∊⍸m]
10↑r ⍝ Just the first 10

Rosalind_1587 Rosalind_6418
 Rosalind_1587 Rosalind_2362
 Rosalind_1587 Rosalind_3930
 Rosalind_1587 Rosalind_0511
 Rosalind_7914 Rosalind_1587
 Rosalind_7914 Rosalind_1264
 Rosalind_2251 Rosalind_3736
 Rosalind_2251 Rosalind_0096
 Rosalind_5406 Rosalind_3099
 Rosalind_5406 Rosalind_1092

## Finding a Shared Motif
http://rosalind.info/problems/lcsm/

All strings in the test set are of equal lengths (1000).

This is a naive brute-force search, made tenable by limiting the substring length to longer than 100 and shorter than 200 letters.

<s>Note: doesn't work on 18.1-pre.</s>

In [198]:
data←FASTA2'rosalind_lcsm.txt'      
substrings ← ⊃,/2↓(⍳∘≢,/¨⊂)⊃data.strings
test←({l←≢⍵⋄(l<200)∧(l>100)}¨substrings)/substrings
test⊃⍨⊃⍸∧/test ∘.(1∊⍷) data.strings  ⍝ ~30s

TGAATGACCCCATGATGGCTGGCAACCCTATTGGTTGAATTTTCAGTTCGACCAAGGTTGAGACCACGTCTTCCCGCACAATATTGTTTTACATCTAGACT

## Independent Alleles
http://rosalind.info/problems/lia/

http://rosalind.info/problems/lia/explanation/

Some more probability maths I can't claim to understand. A binomial distribution B(n, p), with n=2×k (there are two children at each generation) and p=1/4.

In [11]:
lia←{p←{(⍵!2*⍺)×(0.25*⍵)×0.75*(2*⍺)-⍵}⋄⍎3⍕1-+/⍺∘p¨⍳⍵}

In [15]:
assert 0.684=⎕←2 lia 1

0.684

In [14]:
assert 0.178=⎕←7 lia 37

0.178

## Finding a Protein Motif
http://rosalind.info/problems/mprt/

Note: matches may overlap. To get overlapping regex captures, use a capturing group inside a zero-width lookahead assertion. See https://stackoverflow.com/questions/5616822/python-regex-find-all-overlapping-matches

In [38]:
data←file'rosalind_mprt-2.txt'

In [39]:
found←⍬∘≢¨loc←{'(?=(N[^P][ST][^P]))'⎕S 0⊢⍵}¨UniProt data ⍝ TODO: use regex variant for overlapping matches instead

In [40]:
]box off
⍪⊃,/↓⍉↑(found/data) (1+found/loc)

Was OFF

P01878_ALC_MOUSE
 38 99 314 329
 Q7TMB8
 209 291 328 442 607 672 831 858
 Q07287_ZPB_PIG
 70 203 220 333 474
 P07725_CD8A_RAT
 63
 P49286
 4 130
 P06870_KLK1_HUMAN
 102 108 165
 P01044_KNH1_BOVIN
 47 87 168 169 197 204
 P08709_FA7_HUMAN
 205 382
 P01215_GLHA_HUMAN
 76 102
 A2A2Y4
 90 359 407

## Open Reading Frames
http://rosalind.info/problems/orf/

An open reading frame (ORF) is one which starts from the start codon and ends by stop codon, without any other stop codons in between.

In [223]:
codon←0 2 1⍉⍉4 4 4⍴'FLIVFLIVLLIVLLMVSPTASPTASPTASPTAYHNDYHND.QKE.QKECRSGCRSG.RRGWRRG'
dna2rna←'U'@('T'∘=)
revcmpl←{⊖'UAGC'['AUCG'⍳⍵]}
orfs←{'(?=(AUG(...)*?(UAG|UAA|UGA)))'⎕S'\1'⊢⍵}
makeprot←{enc←0 1 2 3['UCAG'⍳¯3↓⍵]⋄⍺[⊂⍤1⊢(⌊3÷⍨≢enc)3⍴enc]}

In [224]:
data←FASTA2'rosalind_orf.txt'
r1←dna2rna (⊃data.strings)
r2←revcmpl r1
s1←orfs r1
s2←orfs r2
r←∪codon∘makeprot¨ s1,s2
{⎕←⍵}¨r

MFGLPKSRSTDVKA
M
MLGKSYSPMVHHPTRLRRI
MVHHPTRLRRI
MSL
MPPLLS
MSPINIARHSRGSSGQRLCCGSSKTSISEDDTLYHKSSSELLTGSALRPGAFNVSINRVYSRDEPNVETMCLPVPS
MSRRCVYLYRRSLRIKCLVAVVRSHSYGSEPLQQGITIGMSSNVSTLANQRSVLDHASEFPGKVMPGGAICHAALTRSNLGWRH
MCLPVPS
MPCCSGSEP
MAPNHCNKA
MSSNVSTLANQRSVLDHASEFPGKVMPGGAICHAALTRSNLGWRH
MRPNSQVR
MPGGAICHAALTRSNLGWRH
MLPSQDPI
MAPLKNLARARYKYCAMYRRKASVLESAQITSTTVVIRYARNRPSKILTK
MYRRKASVLESAQITSTTVVIRYARNRPSKILTK
MCLEAALRVRATAKMARDRVSYNNLFSKDFGWTISRVPYHNSCGSNLGAFENRGLTPVHCAIFISCASQIF
MARDRVSYNNLFSKDFGWTISRVPYHNSCGSNLGAFENRGLTPVHCAIFISCASQIF
MDDFSRTLSQQLWK
MAPS
MTDRPTGHHLTWEFGRMVENAALIG
MVENAALIG
MAPNHCNKAFYAQTTTVQVDTSSRH
MRRLRRYR
MLTLNAPGLKADPVRSSEEDL
MLMGLMIKVGAYAGV
MGLMIKVGAYAGV
MIKVGAYAGV
MPEYEVAS
MKWRLNVTSSYTPQARRVVYHWRVALP
MLQAHIRRRRVGWCTIGE

## Enumerating Gene Orders
http://rosalind.info/problems/perm/

In [21]:
⎕IO←1
pmat←{1≥⍴⍵:↑,↓⍵⋄↑⍪/⍵,∘∇¨⍵∘~¨⍵} ⍝ From dfns: pmat ⍳X
data ← 4
m←pmat ⍳data
{⎕←≢⍵⋄⎕←⍵}m

24
1 2 3 4
1 2 4 3
1 3 2 4
1 3 4 2
1 4 2 3
1 4 3 2
2 1 3 4
2 1 4 3
2 3 1 4
2 3 4 1
2 4 1 3
2 4 3 1
3 1 2 4
3 1 4 2
3 2 1 4
3 2 4 1
3 4 1 2
3 4 2 1
4 1 2 3
4 1 3 2
4 2 1 3
4 2 3 1
4 3 1 2
4 3 2 1

## Calculating Protein Mass
http://rosalind.info/problems/prtm/

In [44]:
data←file'rosalind_prtm.txt'
mmt←71.03711 0.0 103.00919 115.02694 129.04259 147.06841 57.02146 137.05891 113.08406 0.0 128.09496 113.08406 131.04049 114.04293 0.0 97.05276 128.05858 156.10111 87.03203 101.04768 0.0 99.06841 186.07931 0.0 163.06333  0.0

In [45]:
assert 105638.74=⎕←⍎3⍕+/mmt[⎕A⍳⊃data]

105638.74

## Locating Restriction Sites
http://rosalind.info/problems/revp/

We seek the position and length of every reverse palindrome in the string having length between 4 and 12. A reverse palindrome means a substring equal to its reverse complement. 

In [176]:
⎕IO←1
]box off
data←FASTA2'rosalind_revp.txt'
dna←⊃data.strings
A←↓⌿↑(-¯1+⍳≢a) (a←↓(3+⍳9),/⍤0 1⊢dna)                     ⍝ Substrings of increasing lengths
B←↓⌿↑(¯1+⍳≢b) (b←↓⌽(3+⍳9),/⍤0 1⊢{⊖'TAGC'['ATCG'⍳⍵]} dna) ⍝ Substrings of increasing lengths, reverse 
tab←,[0.5]⌿↑A B                                          ⍝ Line up
{⍬≢⍺:⎕←⍺,⍤0 0⊢⍵}⌿↑(⍸¨≡⌿¨tab) (3+⍳9)                      ⍝ Match, find locations, print

Was OFF

1 4
 2 4
 24 4
 36 4
 51 4
 53 4
 60 4
 83 4
 84 4
130 4
152 4
172 4
192 4
231 4
235 4
248 4
268 4
275 4
312 4
322 4
334 4
372 4
383 4
408 4
414 4
465 4
472 4
485 4
510 4
521 4
540 4
568 4
595 4
603 4
626 4
634 4
646 4
650 4
653 4
666 4
670 4
680 4
698 4
723 4
741 4
771 4
785 4
790 4
800 4
807 4
809 4
846 4
851 4
856 4
867 4
868 4
869 4
879 4
880 4
885 4
893 4
915 4
928 4
929 4
 23 6
 35 6
 50 6
 52 6
 59 6
129 6
230 6
311 6
333 6
371 6
413 6
464 6
471 6
649 6
697 6
789 6
867 6
310 8
370 8
463 8
696 8
788 8
866 8
462 10
461 12

## RNA Splicing
http://rosalind.info/problems/splc/
Remove the intron strings, and transcribe the remainder. Regex job.

In [199]:
⎕IO←0
makeprot←{enc←0 1 2 3['TCAG'⍳¯3↓⍵]⋄⍺[⊂⍤1⊢(⌊3÷⍨≢enc)3⍴enc]}
codon←0 2 1⍉⍉4 4 4⍴'FLIVFLIVLLIVLLMVSPTASPTASPTASPTAYHNDYHND.QKE.QKECRSGCRSG.RRGWRRG'
data←FASTA2'rosalind_splc.txt'
codon makeprot (1↓data.strings)⎕R''⊢⊃data.strings

MAPVACRAVHKSPLAGSCELTTLDLRNDTKEVPTSSTSLKPYSLAAQRRISARSHFYDYFYEYLPPLYGRPKLASGANDTLNRSTHKHCLIGELEAGYHRNSQVIYVRSSCRTLTLDLFAVPFPISMMLILITWNVNIARRRRARYMTAVVGYETSFLGKFLQFTLSYPRRWLAMTEESTDLPFANLGS

## Enumerating k-mers Lexicographically
http://rosalind.info/problems/lexf/

In [102]:
data←file'rosalind_lexf.txt'
string←'\s+'⎕R''⊢⊃data
len←⍎⊃1↓data

⍝ Shorter example - comment out to run on a real data set
string←'ACGT'
len←2
⍝ ---------------

↓string[↑,⍳len⍴≢string]

AA AC AG AT CA CC CG CT GA GC GG GT TA TC TG TT

## Longest Increasing Subsequence
http://rosalind.info/problems/lgis/


Here's a tradop implementation of Wikipedia's [pseudocode](https://en.wikipedia.org/wiki/Longest_increasing_subsequence) -- not very "array", sadly.

In [48]:
∇ S←(cmp lss) X;N;P;M;L;i;lo;hi;mid;newL;k
⍝ https://en.wikipedia.org/wiki/Longest_increasing_subsequence
N←≢X
P←N↑¯1
M←(N+1)↑¯1
L←0
:for i :in ⍳N
    (lo hi)←1 L
    :while lo≤hi
        mid←⌈(lo+hi)÷2
        :if X[M[mid]] cmp X[i]
            lo←mid+1
        :else
            hi←mid-1
        :endif
    :endwhile
    newL←lo
    P[i]←M[newL-1]
    M[newL]←i
    :if newL>L
        L←newL
    :endif
:endfor
S←L↑0
k←M[L]
:for i :in ⊖⍳L
    S[i]←X[k]
    k←P[k]
:endfor
∇

In [49]:
data←file'rosalind_lgis.txt'
seq←⍎1⊃data
]box off
10{⍺←0⋄⎕←⍺↑<lss ⍵⋄⎕←⍺↑>lss ⍵}seq  ⍝ NOTE: LONG RESULT; TRUNCATED FOR SPACE 

Was OFF

19 66 171 217 232 320 329 339 464 500
8790 8779 8682 8663 8636 8626 8611 8606 8522 8407

## Genome Assembly as Shortest Superstring
http://rosalind.info/problems/long/

The dataset is guaranteed to satisfy the following condition: there exists a unique way to reconstruct the entire chromosome from these reads by gluing together pairs of reads that overlap by more than half their length.

It was quite hard to understand the intent here -- it does NOT mean pair up, then pair up the pairs etc. It means find a pair - then try to pair this with one of the remaining items.

Not a great solution. 

In [200]:
data←FASTA2'rosalind_long-1.txt'

In [201]:
]dinput
glue←{
    g←{
        ⍺≡⍵:⍬
        m←⍸≡⌿↑(⌽¨,\⌽⍺) (p←,\⍵) ⍝ Perhaps overly inefficient
        ⍬≡m:⍬                  ⍝ No shared
        s←≢p⊃⍨⊢/m              ⍝ Length of longest shared segment
        s≤2÷⍨⍺⌊⍥≢⍵:⍬           ⍝ Shared is less than or equal to half shortest ⍺ ⍵
        ⍺,s↓⍵
    }
    (a b)←(⍺g⍵)(⍵g⍺)           ⍝ Try matching both ways
    (⍬ ⍬)≡a b:⍬                ⍝ No match, either way
    ⍬≡a:b⋄⍬≡b:a                ⍝ One matches
    a<⍥≢b:a⋄b                  ⍝ Both matches; pick shortest
}

In [202]:
]dinput
long←{
    0=≢⍵:⍺
    v←⍸⍬∘≢¨r←⍺∘glue¨⍵
    ⍬≡v:1÷0
    (r⊃⍨⊃v)∇⍵~⍵⌷⍨⊃v
}

In [203]:
join←(⊃data.strings)long 1↓data.strings        ⍝ Around 5s
50↑join ⍝ NOTE: very long (~18k)

AAGTCGATGTAAAACGGGGGGGGGGGGGGCGGCAAACGCACTTGATACCA

## Perfect Matchings and RNA Secondary Structures
http://rosalind.info/problems/pmch/

In [42]:
×/!+/'AC'=⍤0 1⊢'UGCGGGACGCGUCAAGUACCGCGUGCGGACCACCAACCUCUCCCAAAUUUUGUGUAUCCGUGGCGGGCGAUUAAAG'

2.351723106124997067993513984E34

Sadly, Dyalog's lack of bignums makes this annoying. Here's a python version:

```python
from collections import Counter
from math import factorial

seq='UGCGGGACGCGUCAAGUACCGCGUGCGGACCACCAACCUCUCCCAAAUUUUGUGUAUCCGUGGCGGGCGAUUAAAG'
counts = Counter(seq)
factorial(counts["A"])*factorial(counts["C"])
>>>> 23517231061249970679935139840000000
```

It _can_ be done, using the `big` operator from the `dfns` ws:

In [43]:
⎕FR←1287 ⍝ As big as it'll go!
⎕PP←34
'big'⎕CY'dfns' ⍝ Even with ⎕FR←1287 the last product needs to use `×big`

⊃×big/!+/'AC'=⍤0 1⊢'UGCGGGACGCGUCAAGUACCGCGUGCGGACCACCAACCUCUCCCAAAUUUUGUGUAUCCGUGGCGGGCGAUUAAAG'

┌→──────────────────────────────────┐
│23517231061249970679935139840000000│
└───────────────────────────────────┘

## Inferring mRNA from Protein
http://rosalind.info/problems/mrna/

In [50]:
⎕IO←0
data←⊃file'rosalind_mrna.txt'
f←{≢⍸⍵=l}¨a←∪l←'FLIVFLIVLLIVLLMVSPTASPTASPTASPTAYHNDYHND.QKE.QKECRSGCRSG.RRGWRRG'
(1000000|×)/f[a⍳data,'.']

46528

## Partial Permutations
http://rosalind.info/problems/pper/

The number of ways we can pick k items from a set size n is:

    n!/(n−k)!

In [19]:
n←100
k←9
assert 192000=r←⎕←1000000|k(!×∘!⊣)n

192000

You can't submit a solution to this problem before you have solved [Inferring mRNA from Protein](http://rosalind.info/problems/mrna/).

## Enumerating Oriented Gene Orderings
http://rosalind.info/problems/sign/

In [10]:
⎕IO←1
]box on -s=min

n←3
neg←1(⊢+⊣×0=⊢)-1↓⍤1 {⍉2∘⊥⍣¯1⍳2*⍵}n ⍝ All bit patterns of n bits, turn 1 to ¯1 and 0 to 1
sigp←↑,,/(⊂pmat n)⌷⍤1⊢neg×⍤1 1⊢⍳n  ⍝ For every possible negation of ⍳n, pick the permutations
{⎕←print ≢⍵⋄⎕←print ⍵}sigp         ⍝ Format for output

⍝ For n>3, write to a file
⍝(⍕≢sigp) ⎕NPUT '/Users/stefan/ros.txt'1
⍝(sigp ⎕CSV⍠'Separator' ' '⊢'')⎕NPUT '/Users/stefan/ros.txt'2

Was ON -style=min

48
1 2 -3 
1 -3 2 
2 1 -3 
2 -3 1 
-3 1 2 
-3 2 1 
1 -2 3 
1 3 -2 
-2 1 3 
-2 3 1 
3 1 -2 
3 -2 1 
1 -2 -3 
1 -3 -2 
-2 1 -3 
-2 -3 1 
-3 1 -2 
-3 -2 1 
-1 2 3 
-1 3 2 
2 -1 3 
2 3 -1 
3 -1 2 
3 2 -1 
-1 2 -3 
-1 -3 2 
2 -1 -3 
2 -3 -1 
-3 -1 2 
-3 2 -1 
-1 -2 3 
-1 3 -2 
-2 -1 3 
-2 3 -1 
3 -1 -2 
3 -2 -1 
-1 -2 -3
-1 -3 -2
-2 -1 -3
-2 -3 -1
-3 -1 -2
-3 -2 -1
1 2 3 
1 3 2 
2 1 3 
2 3 1 
3 1 2 
3 2 1

## Finding a Spliced Motif
http://rosalind.info/problems/sseq/

This is sort of putting the match matrix `needle ∘.= haystack` into row echelon form.

In [187]:
⎕IO←1
data←FASTA2'rosalind_sseq-1.txt'
haystack←⊃data.strings
needle←⊃⊢/data.strings

In [188]:
]dinput
sseq←{
    ¯1 {
        0=≢⍵:⊖¯1↓⍺
        (⍺,⍨⊃(m>⊃⍺)/m←⍸⊃⍵)∇1↓⍵
    } ↓⍺∘.=⍵
}

In [189]:
⊢r←needle sseq haystack
assert r≡3 4 5 8 9 10 12 13 14 16 27 34 35 40 43 52 53 54 56 59 61 63 64 69 72 77 88 98 101

3 4 5 8 9 10 12 13 14 16 27 34 35 40 43 52 53 54 56 59 61 63 64 69 72 77 88 98 101

## Transitions and Transversions
http://rosalind.info/problems/tran/

In [192]:
⎕IO←0
data←FASTA2'rosalind_tran-1.txt'
(a b c)←+/1 2 3∘.=|-⌿'ACGT'⍳↑data.strings
b÷a+c

2.202380952380952380952380952380952

## Completing a Tree
http://rosalind.info/problems/tree/

Find the number of connected components in a graph. The traditional approach is to take a vertex, and apply a breadth-first-search to find all reachable nodes, and remove those from the graph until the graph is empty. It is possible that there is a smarter, more "array" approach I can't see.

We borrow the bfs routine from the `dfns` workspace.

Index origin of 1 matches the problem description's vertex labeling.

Find the number of connected components in a graph, where the graph is represented as an adjacency index vector.

In [137]:
]dinput
ccon←{⎕IO←1
    graph←⍵ ⋄ v←⍳≢⍵
    bfs←{g←⍺⋄⍬{⍵≡⍬:⍺⋄h←1↑⍵⋄(⍺,h)∇⍺~⍨(1↓⍵)∪h⊃g}⍵} ⍝ Adapted from http://dfns.dyalog.com/n_bfs.htm
    0 {⍬≡s←v~⍵:⍺⋄(⍺+1)∇⍵,graph bfs ⊃s} ⍬ ⍝ Accumulate connected components to the left, seen vertices to the right
}

In [138]:
data←⍎¨⊃⎕NGET '/Users/stefan/work/dyalog/rosalind/data/rosalind_tree.txt'1

In [139]:
vtxC←⊃data
adjM←vtxC vtxC⍴0
adjM[1↓data]←1

We need to mirror the adjacency matrix as the graph is undirected. Then convert to an adjacency index vector, following the notes given in the section on graphs in [dfns workspace](http://dfns.dyalog.com/n_Graphs.htm).

In [140]:
adjV←{{⍵/⍳⍴⍵}¨↓⍵}+⌿↑adjM (⍉adjM)

We seek number of edges to add, which is one less than the number of connected components

In [141]:
⊢r←¯1+ccon adjV 
assert 64=r

64

An alternative solution is to use the `scc` function from `dfns` which I discovered after I wrote the above. This is an implementation of [Tarjan's algorithm](https://en.wikipedia.org/wiki/Tarjan%27s_strongly_connected_components_algorithm) for finding the strongly connected components of a graph:

In [146]:
'scc'⎕CY'dfns' ⍝ Strongly Connected Components http://dfns.dyalog.com/n_scc.htm

In [147]:
¯1+≢∪scc adjV 

64

and finally, from ninja master @ngn, using the transitive closure:

In [150]:
¯1+≢∪{(∪⍳⊢)↓∧∘⍉⍨∨.∧⍨⍣≡i∘.∊⍵,¨i←⍳≢⍵} adjV

64

@ngn notes:

> The version is very good for small graphs but its space and time requirements
grow rapidly as the size increases.

## Error Correction in Reads
http://rosalind.info/problems/corr/


```apl
data←FASTA2'rosalind_corr-3.txt'
a←corr data.strings
(⊂a) ⎕NPUT '/Users/stefan/work/notebooks/rosalind/data/CORR.txt'1
```

In [147]:
]dinput
corr←{
    (a b)←{↓⍉{⍺,≢⍵}⌸⍵} all←⍵,{⊖'TAGC'['ATCG'⍳⍵]}¨⍵   ⍝ Add revcmp and create a frequency table
    right←all~wrong←c↑⍨⌊2÷⍨≢c←a/⍨1=b                 ⍝ Pick candidates present as-is in the data
    hd1←1=wrong∘.(+/≠)right                          ⍝ Hamming distance = 1
    hd1[⍸2>+/hd1;]←0                                 ⍝ Matches need to occur at least twice in the data.
    first←<\⍤1⊢hd1                                   ⍝ Only consider first match
    {⍺,'->',⍵}/wrong,⍪first/⍤1 1⊢right
}

In [148]:
data←FASTA2'rosalind_corr-3.txt'
a←corr data.strings
↑10↑a ⍝ big data...

AGTAATCTTAAAGCGCACACAACATCCTACACAGACTCTATTGCCGGGAC->AGTAATCTTAAAGCGCACACAACATCCGACACAGACTCTATTGCCGGGAC
CACAACATCCGACACAGACTCTATTGCCGGGATTAAGGACGGTATTACAT->CACAACATCCGACACAGACTCTATTGCCGGGACTAAGGACGGTATTACAT
CACACAACATCCGACCCAGACTCTATTGCCGGGACTAAGGACGGTATTAC->CACACAACATCCGACACAGACTCTATTGCCGGGACTAAGGACGGTATTAC
GCATGCAGACAAGCTCAAATGCGTACACCATAGCCCTGTTCACAAAGAAA->ACATGCAGACAAGCTCAAATGCGTACACCATAGCCCTGTTCACAAAGAAA
AGGGTTCTTGTGTCGTAAGTAATCTTAAAGCGCACACAACATCCGACACA->AGGGTTCTTGGGTCGTAAGTAATCTTAAAGCGCACACAACATCCGACACA
AATCTTAAAGCGCACACAACATCCGACACAGACTCTATTGCCGCGACTAA->AATCTTAAAGCGCACACAACATCCGACACAGACTCTATTGCCGGGACTAA
TAATCTTAAAGCGCACACAACATCCGACACAGACTCTATTGCGGGGACTA->TAATCTTAAAGCGCACACAACATCCGACACAGACTCTATTGCCGGGACTA
CTTGGGTCGTAAGTAATCTTACAGCGCACACAACATCCGACACAGACTCT->CTTGGGTCGTAAGTAATCTTAAAGCGCACACAACATCCGACACAGACTCT
AGACAGGGTTCTTGGGTCGTAAGTAATCTTAAAGCGCACACAACCTCCGA->AGACAGGGTTCTTGGGTCGTAAGTAATCTTAAAGCGCACACAACATCCGA
TAATCTCAAAGCGCACACAACATCCGACACAGACTCTATTGCCGGGACTA->TAATCTTAAAGCGCACACAACATCCGACACAGACTCTATTGCCGGGACTA

## Finding a Shared Spliced Motif
http://rosalind.info/problems/lcsq/

In [193]:
]dinput
lcsq←{
    M←↑⊃{⍵,⊂⌈\last⌈(⍺+0,¯1↓last←⊃⌽⍵)⌈0,¯1↓⍺}/(⊂⊂{0}¨⍵),⍨⌽↓⍺∘.=⍵
    mat←2<⌿2</0,M
    pos←(⍸mat){⍵[⍋⍵]⊆⍺[⍋⍵]}mat/⍥,1↓M
    ind←1↓¨{⍵,¯1↑⍺/⍨∧/¨⍺<¯1↑⍵}/pos,⊂⊂⍴M
    ⌽⍺⌷⍨⊃¨¨ind
}

In [194]:
data←FASTA2'rosalind_lcsq-2.txt'
⊃lcsq/data.strings

TCCCAAGCTGAACCGTTGGCACGAACCCATGTGAGTCAGGACCCAAATCGTCCACATAAATTTGGTTGCATGCTGCCCCTCGGCCCCTATAGGCAATTTCTGGTGGGGTCACAAGTTGGAGCACTAGTAAGTGGTTACCATTACGCCCTGTGAAGGATAAAGGTCGCAAATTAAACCTCGGCATGTCGTACCGTGTTTGTCTAACACACGCGGATCACAGCATCTTATAACAGATCGGACTCACCCTGGCTTCGCCAAAAAAGCCCGATGACTGCCGAGGGCCATCGGGCTGCAACAATAACTCGGTATTGACCATTTGTGGACGACAGAATTTTCTGGGAGTGGTATCGCGGTGGCGTCGGCGCGCACTGGATAGGACATTGTGGAAGTCTTTCCAGCCGCTCCCTTTTAAAACGTAGGTTGCGTTCAATAATCCATCTCACCGATCCCTGAGCCCTATTGAAGTGAGGGTATTTCATCTGATCTTGCAAGGAGCGGACGGAAAATGGGACTTCTACGTGCGAACATCATTTCAACTTCTACAATACAATCCTAGCGT

## Calculating Expected Offspring
http://rosalind.info/problems/iev/

Six possible genotype pairings:

1. AA - AA
1. AA - Aa
1. AA - aa
1. Aa - Aa
1. Aa - aa
1. aa - aa

Probability that offspring has `A`: 1 1 1 0.75 0.5 0

So we multiply each probability with the given number of couples per pairing type, sum, and multiply by 2, as we're looking at 2 offspring per couple.

In [10]:
data ← 19083 17341 19657 16896 16197 18256
iev ← {2×⍵+.×1 1 1 0.75 0.5 0}
assert 153703=⎕←iev data

153703

## Introduction to Random Strings
http://rosalind.info/problems/prob/

In [16]:
]box off
data←⊃⎕NGET'/Users/stefan/work/notebooks/rosalind/data/rosalind_prob.txt'1

Was ON

In [17]:
prob←{∆←at cg (cg←⍺÷2) (at←2÷⍨1-⍺)⋄⍎3⍕10⍟×/∆['ACGT'⍳⍵]}

In [18]:
s←⊃data ⋄ A←⍎1⊃data
'-'@(=∘'¯')⍕A prob⍤0 1⊢s ⍝ Remember to flip APL high minus to dash

-80.361 -77.242 -65.164 -61.852 -60.076 -55.956 -54.853 -53.654 -53.314 -53.223 -53.326 -53.993 -57.01 -59.232 -62.765 -67.267

## Counting Phylogenetic Ancestors
http://rosalind.info/problems/inod/

In an unrooted binary tree with `n` leaves, there are `n−2` internal nodes, for a total of `2n−2` nodes and, therefore, `2n−1` edges.

In [19]:
n←3994
n-2

3992

## k-Mer Composition
http://rosalind.info/problems/kmer/

We don't need to do any sorting. We generate the possible kmers in lexicographical order:

```apl
'ACGT'[↑,⍳4 4 4 4]
```
The k-mers in the string are generated with a windowed reduction `4,/⍵`, and we bind these to dyadic `⍳` to pre-hash the array for lookups with `k∘⍳`.

In [152]:
data←FASTA2'rosalind_kmer-1.txt'

In [153]:
kmer←{(k c)←↓⍉{⍺,≢⍵}⌸4,/⍵⋄(c,0)[k∘⍳↓'ACGT'[↑,⍳4/4]]}

In [154]:
kmer ⊃data.strings

351 366 363 390 398 388 363 385 412 378 402 388 404 373 424 354 389 394 402 398 396 387 405 369 401 380 341 380 388 389 379 444 427 366 389 382 373 391 399 398 402 394 387 373 425 379 395 363 360 368 388 408 401 406 373 405 394 381 389 425 398 388 384 415 363 401 397 350 412 388 395 390 400 391 398 393 367 387 399 368 398 391 415 368 399 371 404 379 392 419 428 393 397 416 389 362 403 393 356 407 373 404 347 397 395 363 356 416 389 397 359 381 401 402 397 403 382 405 397 394 400 418 348 418 418 400 405 364 380 388 429 416 386 388 376 413 382 395 355 383 362 432 363 449 361 396 375 357 376 412 408 401 369 351 383 378 409 382 401 386 370 413 389 407 374 377 348 370 399 366 390 374 361 389 387 378 400 385 383 401 388 410 379 420 392 400 373 393 383 369 405 359 376 379 390 399 387 393 368 412 370 397 401 398 391 393 403 414 363 404 390 398 401 383 415 415 397 371 378 375 409 391 415 395 413 391 381 410 369 425 388 413 383 346 396 352 394 432 417 394 383 405 398 389 384 393 372 391 409 396 367 401 376 383 379 381

## Speeding Up Motif Finding

http://rosalind.info/problems/kmp/

Knuth-Morris-Pratt. A beautiful thing.

In [195]:
⎕IO←0
data←FASTA2'rosalind_kmp-1.txt'

In [156]:
]dinput
P←prefix s;m;i;j
m←≢s
P←0⍴⍨m
j←0
:for i :in 1+⍳¯1+m
    :while j≥0
        :if s[j]=s[i]
            :leave
        :endif
        :if (j-1)≥0
            j←P[j-1]
        :else
            j←¯1
        :endif
    :endwhile
    j+←1
    P[i]←j
:endfor

In [161]:
result ← prefix ⊃data.strings

In [162]:
assert 0 1 2 0 1 2 3 3 3 3≡⎕←10↑result ⍝ VERY long!

0 1 2 0 1 2 3 3 3 3

Here's a dfn version of the same:

In [163]:
]dinput
pfx←{
    s←⍵
    P←0⍴⍨≢⍵
    j←0
    0{
        ⍺=≢⍵:P
        i←⍺⊃⍵
        P[i]←j⊢←1+{⍵<0:⍵ ⋄ s[⍵]=s[i]:⍵ ⋄ 0≤⍵-1:∇ P[⍵-1] ⋄ ¯1}j
        (⍺+1)∇ ⍵
    }1+⍳¯1+≢⍵
}

In [164]:
result ← pfx ⊃data.strings
assert 0 1 2 0 1 2 3 3 3 3≡⎕←10↑result ⍝ VERY long!

0 1 2 0 1 2 3 3 3 3

## Inferring Protein from Spectrum
http://rosalind.info/problems/spec/

In [84]:
data ←⍎¨file'rosalind_spec-1.txt'
mmt←71.0371 0 103.0092 115.0269 129.0426 147.0684 57.0215 137.0589 113.0841 0 128.095 113.0841 131.0405 114.0429 0 97.0528 128.0586 156.1011 87.032 101.0477 0 99.0684 186.0793 0 163.0633 0

In [85]:
diff←{⍎4⍕⍵}¨|2-/data ⍝ 4 decimal places

In [86]:
⎕A[mmt⍳diff]

VRNDITISGMAWIVNPCWFCDQTQDCIEPSEYGQCGWEDIEWIIYDYFWPSTFIEACDKIDHGPDIEYYHRRNAQNMERKRWPIWGGWAI

## Ordering Strings of Varying Length Lexicographically
https://rosalind.info/problems/lexv/

In [105]:
⎕IO←0
one←'DNA '[↑,(⍳3),¨⊂3 3]
two←'DNA '[↑,(⍳3 3),¨3]
three←'DNA '[↑,⍳3 3 3]
s←three⍪two⍪one
s[' DNA'⍋s;]

D 
DD 
DDD
DDN
DDA
DN 
DND
DNN
DNA
DA 
DAD
DAN
DAA
N 
ND 
NDD
NDN
NDA
NN 
NND
NNN
NNA
NA 
NAD
NAN
NAA
A 
AD 
ADD
ADN
ADA
AN 
AND
ANN
ANA
AA 
AAD
AAN
AAA

In [112]:
⎕IO←1

In [123]:
data←'DBREQFKCMIAL'
len←3
s←↑len {⊃,/(⍳⍺){,↓⍵[↑,¨⍳⍺⍴≢⍵]}¨⊂⍵} data
out←s[s⍋⍨' ',data;]
(⊂↓out)⎕NPUT'/Users/stefan/work/notebooks/rosalind/data/lexv-out.txt'1

## Counting Subsets
https://rosalind.info/problems/sset/

Boils down to how to do modular exponentiation without overflow. Solution in APLCart.

In [3]:
modexp←{⍺⍺{⍺⍺|⍺×⍺⍺|×⍨⍵}⌿⍺*⍤1⊖0⍪2⊥⍣¯1⊢⍵} ⍝ https://aplcart.info?q=modulo%20power

In [6]:
assert 483392=⎕←2 (1e6 modexp) 873

483392

## Mortal Fibonacci Rabbits
https://rosalind.info/problems/fibd/

A small modification to the normal recurrence to only sum the 'living' rabbit pairs.

In [11]:
⎕PP←34   ⍝ Numeric print precision
⎕FR←1287 ⍝ IEEE 754-2008 128-bit decimal floating-point operations

In [19]:
]dinput
fibn←{ ⍝ Ungolfed
    (m n)←⍺ ⍵
    {
        ⍺←1 1
        ⍵=n-2:⍺
        (⍺,⍺{m>⍵+2:+⌿⍺[⍵+0 1]⋄+⌿⍺[⍵-⍳m-1]}⍵)∇⍵+1
    } 0
}

In [17]:
fibn2←{⍺←1 1⋄⍵=⍵⍵-2:⍺⋄(⍺,⍺(⍺⍺{⍺⍺>⍵+2:+⌿⍺[⍵+0 1]⋄+⌿⍺[⍵-⍳⍺⍺-1]})⍵)∇⍵+1}

In [18]:
assert 4=⎕←⊃⊖(3 fibn2 6) 0
assert 258314806822396236=⎕←⊃⊖(18 fibn2 85) 0

4

258314806822396236

## Introduction to Pattern Matching
https://rosalind.info/problems/trie/

Flat, fairly fast (~35ms). Hard to get right, sadly.

In [64]:
]dinput
trie←{⎕IO←0
    n←,'⍣'                          ⍝ Nodes. ⍣ is the root placeholder
    p←,0                            ⍝ Parents
    mk←{p,←⍺⋄i←≢n⋄n,←⍵⋄i}           ⍝ Make a new child node, return its index in n
    _←0 {                           ⍝ Function to add the letters of a word to the trie, starting at node 0
        0=≢⍵:⍬
        c←0~⍨⍸p=⍺                   ⍝ Find ⍺'s children: nodes where ⍺ is the parent
        0≠+⌿m←n[c]=⊃⍵:(c⊃⍨⊃⍸m)∇1↓⍵  ⍝ If first letter matches one of the children, recur down that branch
        (⍺ mk ⊃⍵)∇1↓⍵               ⍝ Else, attach a new edge here
    }¨⊆⍵
    ⍉↑(1+1↓p)(2+⍳≢nn)(nn←1↓n)       ⍝ Return as adjacency list, numbered as indicated by problem statement
}

In [63]:
trie 'ATAGA' 'ATC' 'GAT'            ⍝ Test case given

┌→─────┐
↓1 2 A│
│2 3 T│
│3 4 A│
│4 5 G│
│5 6 A│
│3 7 C│
│1 8 G│
│8 9 A│
│9 10 T│
└+─────┘

In [56]:
data←file 'rosalind_trie.txt'

In [57]:
result←trie data

In [58]:
⍴result

┌→─────┐
│8740 3│
└~─────┘

In [59]:
20↑result

┌→──────┐
↓ 1 2 T│
│ 2 3 G│
│ 3 4 G│
│ 4 5 T│
│ 5 6 A│
│ 6 7 G│
│ 7 8 G│
│ 8 9 A│
│ 9 10 T│
│10 11 G│
│11 12 T│
│12 13 G│
│13 14 A│
│14 15 G│
│15 16 T│
│16 17 T│
│17 18 G│
│18 19 G│
│19 20 T│
│20 21 A│
└+──────┘